# 第10章A：PyTorch基础入门

> **从NumPy手写神经网络到深度学习框架的桥梁**

---

## 🎯 学习目标

完成本章后，你将能够：

1. **理解为什么需要深度学习框架**：对比NumPy手写实现的痛点
2. **掌握PyTorch Tensor**：张量创建、运算、与NumPy对比
3. **理解自动微分Autograd**：告别手写反向传播
4. **使用nn.Module定义模型**：对比NumPy版MLP与PyTorch版MLP

---

## 📚 本系列章节

| 章节 | 主题 | 内容 |
|------|------|------|
| **10a** | PyTorch基础 | Tensor、Autograd、nn.Module |
| 10b | PyTorch训练 | 损失函数、优化器、完整训练循环、实战项目 |
| 10c | TensorFlow/Keras | Keras API、框架对比、总结练习 |

---

# Part 1：为什么需要深度学习框架？

---

## 1.1 回顾：NumPy实现的痛点

在前9章中，我们从零用NumPy实现了完整的神经网络：

- ✅ **感知机和MLP**（第1-3章）
- ✅ **反向传播算法**（第4章）
- ✅ **损失函数和优化器**（第5章）
- ✅ **正则化技术**（第6-7章）
- ✅ **权重初始化和训练技巧**（第8-9章）

### 这很重要，但也很痛苦...

让我们回顾一下手写反向传播有多复杂：

In [ ]:
import numpy as np

# ============================================================
# 回顾：NumPy从零实现的神经网络（来自第3-4章）
# ============================================================
# 这是我们之前必须手写的代码...

class NumpyMLP:
    """
    NumPy实现的多层感知机
    
    结构: 输入层 -> 隐藏层(ReLU) -> 输出层(Softmax)
    
    痛点：
    1. 必须手动推导每一层的梯度公式
    2. 必须手动实现链式法则
    3. 容易出错，调试困难
    4. 无法使用GPU加速
    """
    
    def __init__(self, input_size, hidden_size, output_size):
        # ----- 手动初始化权重（Xavier初始化） -----
        # 需要自己实现初始化策略
        self.W1 = np.random.randn(input_size, hidden_size) * np.sqrt(2.0 / input_size)
        self.b1 = np.zeros(hidden_size)
        self.W2 = np.random.randn(hidden_size, output_size) * np.sqrt(2.0 / hidden_size)
        self.b2 = np.zeros(output_size)
        
    def relu(self, x):
        """ReLU激活函数：需要手动实现"""
        return np.maximum(0, x)
    
    def relu_derivative(self, x):
        """ReLU导数：反向传播需要"""
        return (x > 0).astype(float)
    
    def softmax(self, x):
        """Softmax函数：需要手动实现数值稳定版本"""
        exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
        return exp_x / np.sum(exp_x, axis=1, keepdims=True)
    
    def forward(self, X):
        """
        前向传播
        
        必须手动保存中间结果用于反向传播！
        """
        # 第一层：线性变换 + ReLU
        self.z1 = X @ self.W1 + self.b1        # 保存z1用于反向传播
        self.a1 = self.relu(self.z1)           # 保存a1用于反向传播
        
        # 第二层：线性变换 + Softmax
        self.z2 = self.a1 @ self.W2 + self.b2  # 保存z2
        self.a2 = self.softmax(self.z2)        # 输出概率
        
        return self.a2
    
    def backward(self, X, y_true, y_pred):
        """
        反向传播 - 这是最痛苦的部分！
        
        必须手动推导每个参数的梯度：
        - 需要理解链式法则
        - 需要处理矩阵求导
        - 容易出错！
        
        参数:
            X: 输入数据, shape (batch_size, input_size)
            y_true: 真实标签(one-hot), shape (batch_size, output_size)
            y_pred: 预测概率, shape (batch_size, output_size)
        """
        m = X.shape[0]  # batch size
        
        # ----- 输出层梯度 -----
        # dL/dz2 = y_pred - y_true（交叉熵+Softmax的简化形式）
        dz2 = y_pred - y_true  # (batch_size, output_size)
        
        # dL/dW2 = a1^T @ dz2
        # 这里需要理解矩阵求导的转置规则！
        self.dW2 = (self.a1.T @ dz2) / m
        
        # dL/db2 = sum(dz2, axis=0)
        self.db2 = np.sum(dz2, axis=0) / m
        
        # ----- 隐藏层梯度（链式法则继续） -----
        # dL/da1 = dz2 @ W2^T
        da1 = dz2 @ self.W2.T
        
        # dL/dz1 = dL/da1 * relu'(z1)
        # 必须保存z1才能计算ReLU导数！
        dz1 = da1 * self.relu_derivative(self.z1)
        
        # dL/dW1 = X^T @ dz1
        self.dW1 = (X.T @ dz1) / m
        
        # dL/db1 = sum(dz1, axis=0)
        self.db1 = np.sum(dz1, axis=0) / m
    
    def update_parameters(self, learning_rate):
        """
        手动更新参数
        
        这只是最简单的SGD！
        如果要实现Adam，还需要额外50+行代码...
        """
        self.W1 -= learning_rate * self.dW1
        self.b1 -= learning_rate * self.db1
        self.W2 -= learning_rate * self.dW2
        self.b2 -= learning_rate * self.db2


print("="*60)
print("  NumPy实现的痛点总结")
print("="*60)
print("""
❌ 痛点1：手动推导梯度公式
   - 每增加一层，就要推导新的梯度
   - 矩阵求导容易出错

❌ 痛点2：手动保存中间变量
   - forward必须保存z1, a1等
   - backward才能用来计算梯度

❌ 痛点3：手动实现优化器
   - SGD简单，但Adam需要50+行代码
   - 还有AdaGrad, RMSprop等...

❌ 痛点4：无GPU加速
   - NumPy只能用CPU
   - 大规模数据训练很慢

❌ 痛点5：代码难以维护
   - 网络结构改变需要重写backward
   - 不同激活函数需要不同的导数实现
""")

## 1.2 深度学习框架解决了什么问题？

### 🚀 框架的核心价值

| 痛点 | NumPy手写 | 框架解决方案 |
|------|-----------|-------------|
| **梯度计算** | 手动推导每层公式 | **自动微分(Autograd)** |
| **中间变量** | 手动保存a1, z1等 | **计算图自动记录** |
| **优化器** | 每个优化器50+行代码 | **一行调用** `optim.Adam()` |
| **GPU加速** | 不支持 | **`.to(device)`** 自动转移 |
| **代码维护** | 改结构就要重写backward | **只需修改forward** |

---

### 🔥 两大主流框架对比

| 特性 | PyTorch | TensorFlow/Keras |
|------|---------|------------------|
| **开发者** | Meta (Facebook) | Google |
| **编程风格** | 动态图（Pythonic） | Keras高层API简洁 |
| **学术研究** | ⭐⭐⭐⭐⭐ 主流 | ⭐⭐⭐ |
| **工业部署** | ⭐⭐⭐⭐ | ⭐⭐⭐⭐⭐ 成熟 |
| **调试体验** | 像Python一样调试 | TF2.0后改进很大 |
| **学习曲线** | 适中 | Keras极简，TF底层复杂 |

**本系列策略：**
- **10a-10b**：深入学习 **PyTorch**（学术界首选，代码更接近原理）
- **10c**：快速上手 **Keras**（工业界常用，了解高层API便利性）

In [1]:
# ============================================================
# 环境检测：检查PyTorch安装和GPU支持
# ============================================================

import sys
import platform

print("="*60)
print("  深度学习环境检测")
print("="*60)

# ----- 系统信息 -----
print(f"\n📱 系统信息:")
print(f"   操作系统: {platform.system()} {platform.machine()}")
print(f"   Python版本: {sys.version.split()[0]}")

# ----- PyTorch检测 -----
print("\n" + "-"*60)
try:
    import torch
    print(f"✅ PyTorch版本: {torch.__version__}")
    
    # 检测加速设备
    # CUDA（NVIDIA GPU）
    if torch.cuda.is_available():
        device = torch.device("cuda")
        print(f"   🎮 CUDA可用: GPU = {torch.cuda.get_device_name(0)}")
    # MPS（Apple Silicon GPU）
    elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
        device = torch.device("mps")
        print(f"   🍎 MPS可用: Apple Silicon GPU加速")
        # 测试MPS是否正常
        try:
            _ = torch.randn(100, 100, device=device) @ torch.randn(100, 100, device=device)
            print(f"   MPS测试: ✓ 正常工作")
        except Exception as e:
            print(f"   MPS测试: ✗ 回退到CPU")
            device = torch.device("cpu")
    else:
        device = torch.device("cpu")
        print(f"   GPU加速: 不可用（使用CPU）")
    
    print(f"\n   📍 推荐设备: {device}")
    PYTORCH_AVAILABLE = True
    
except ImportError:
    print("❌ PyTorch未安装")
    print("   安装命令: pip install torch torchvision")
    PYTORCH_AVAILABLE = False
    device = None

# ----- 通用库 -----
print("\n" + "-"*60)
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# 绘图设置
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'SimHei', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['figure.figsize'] = [10, 6]

print("✅ 通用库加载完成 (numpy, matplotlib)")

# ----- 设备选择辅助函数 -----
def get_device():
    """
    自动选择最佳计算设备
    
    优先级: CUDA > MPS > CPU
    
    使用示例:
        device = get_device()
        model = model.to(device)
        data = data.to(device)
    """
    if torch.cuda.is_available():
        return torch.device("cuda")
    elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
        return torch.device("mps")
    return torch.device("cpu")

print("\n" + "="*60)
print("✅ 环境准备完成！")
print("="*60)

  深度学习环境检测

📱 系统信息:
   操作系统: Darwin arm64
   Python版本: 3.8.18

------------------------------------------------------------
✅ PyTorch版本: 2.4.1
   🍎 MPS可用: Apple Silicon GPU加速
   MPS测试: ✓ 正常工作

   📍 推荐设备: mps

------------------------------------------------------------
✅ 通用库加载完成 (numpy, matplotlib)

✅ 环境准备完成！


---

# Part 2：PyTorch基础

---

## 2.1 Tensor张量：比NumPy更强的多维数组

### 什么是Tensor？

Tensor（张量）是PyTorch的核心数据结构：

- **本质**：多维数组（类似NumPy的ndarray）
- **优势1**：可以在GPU上运行
- **优势2**：支持自动求导（Autograd）

### 维度对应

| 维度 | 名称 | 示例 |
|------|------|------|
| 0维 | 标量(scalar) | 损失值 3.14 |
| 1维 | 向量(vector) | 偏置 [0.1, 0.2, 0.3] |
| 2维 | 矩阵(matrix) | 权重 W (128, 64) |
| 3维 | 3阶张量 | 批量图像 (batch, height, width) |
| 4维 | 4阶张量 | 彩色图像 (batch, channels, H, W) |

In [2]:
import torch

print("="*60)
print("  2.1 张量创建")
print("="*60)

# ============================================================
# 1. 创建张量的多种方式
# ============================================================
print("\n1️⃣ 创建张量的多种方式:")

# ----- 从Python列表创建 -----
# 这是最直观的方式，类似 np.array()
x_list = torch.tensor([1, 2, 3, 4])
print(f"\n从列表创建: {x_list}")
print(f"  形状(shape): {x_list.shape}")
print(f"  数据类型(dtype): {x_list.dtype}")
print(f"  设备(device): {x_list.device}")

# ----- 指定数据类型 -----
# 神经网络通常使用float32
x_float = torch.tensor([1, 2, 3], dtype=torch.float32)
print(f"\n指定float32类型: {x_float}, dtype={x_float.dtype}")

# ----- 创建特殊张量 -----
print("\n创建特殊张量:")

# 全零张量（常用于初始化偏置）
zeros = torch.zeros(2, 3)  # 2x3矩阵
print(f"  全零 (2,3):\n{zeros}")

# 全一张量
ones = torch.ones(3, 4)
print(f"\n  全一 (3,4):\n{ones}")

# 单位矩阵（常用于初始化某些权重）
eye = torch.eye(3)  # 3x3单位矩阵
print(f"\n  单位矩阵 (3,3):\n{eye}")

# ----- 随机张量 -----
print("\n创建随机张量:")

# 标准正态分布 N(0, 1)（最常用于权重初始化）
randn = torch.randn(2, 3)
print(f"  正态分布N(0,1):\n{randn}")

# 均匀分布 U(0, 1)
rand = torch.rand(2, 3)
print(f"\n  均匀分布U(0,1):\n{rand}")

# 指定范围的整数
randint = torch.randint(low=0, high=10, size=(2, 3))
print(f"\n  随机整数[0,10):\n{randint}")

  2.1 张量创建

1️⃣ 创建张量的多种方式:

从列表创建: tensor([1, 2, 3, 4])
  形状(shape): torch.Size([4])
  数据类型(dtype): torch.int64
  设备(device): cpu

指定float32类型: tensor([1., 2., 3.]), dtype=torch.float32

创建特殊张量:
  全零 (2,3):
tensor([[0., 0., 0.],
        [0., 0., 0.]])

  全一 (3,4):
tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]])

  单位矩阵 (3,3):
tensor([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]])

创建随机张量:
  正态分布N(0,1):
tensor([[-0.3815,  0.4283,  0.9874],
        [ 0.2134,  0.0991,  0.6444]])

  均匀分布U(0,1):
tensor([[0.5992, 0.8686, 0.0689],
        [0.5072, 0.9290, 0.3112]])

  随机整数[0,10):
tensor([[2, 1, 5],
        [5, 0, 8]])


In [3]:
print("="*60)
print("  2.1 张量与NumPy对比")
print("="*60)

# ============================================================
# 2. Tensor vs NumPy ndarray：几乎一样的操作！
# ============================================================
print("\n2️⃣ Tensor与NumPy ndarray对比:")

import numpy as np

# 创建相同数据
np_array = np.array([[1, 2, 3], [4, 5, 6]], dtype=np.float32)
torch_tensor = torch.tensor([[1, 2, 3], [4, 5, 6]], dtype=torch.float32)

print(f"\nNumPy array:\n{np_array}")
print(f"\nPyTorch tensor:\n{torch_tensor}")

# ----- 操作对比表 -----
print("\n" + "-"*60)
print("常用操作对比:")
print("-"*60)
print(f"{'操作':<20} {'NumPy':<25} {'PyTorch':<25}")
print("-"*60)
print(f"{'创建零矩阵':<20} {'np.zeros((2,3))':<25} {'torch.zeros(2,3)':<25}")
print(f"{'创建随机矩阵':<20} {'np.random.randn(2,3)':<25} {'torch.randn(2,3)':<25}")
print(f"{'矩阵乘法':<20} {'A @ B 或 np.dot(A,B)':<25} {'A @ B 或 torch.mm(A,B)':<25}")
print(f"{'逐元素乘法':<20} {'A * B':<25} {'A * B':<25}")
print(f"{'求和':<20} {'np.sum(A)':<25} {'torch.sum(A) 或 A.sum()':<25}")
print(f"{'形状':<20} {'A.shape':<25} {'A.shape 或 A.size()':<25}")
print(f"{'转置':<20} {'A.T':<25} {'A.T':<25}")
print(f"{'reshape':<20} {'A.reshape(3,2)':<25} {'A.reshape(3,2) 或 view':<25}")
print("-"*60)

# ----- 相互转换 -----
print("\n3️⃣ NumPy与Tensor相互转换:")

# NumPy -> Tensor
np_data = np.array([1.0, 2.0, 3.0])
tensor_from_np = torch.from_numpy(np_data)  # 共享内存！
tensor_copy = torch.tensor(np_data)          # 拷贝数据

print(f"\n原始NumPy: {np_data}")
print(f"torch.from_numpy(): {tensor_from_np}  ⚠️ 共享内存")
print(f"torch.tensor(): {tensor_copy}  ✓ 独立拷贝")

# 验证内存共享
np_data[0] = 999
print(f"\n修改NumPy[0]=999后:")
print(f"  from_numpy版本也变了: {tensor_from_np}")
print(f"  tensor版本不变: {tensor_copy}")

# Tensor -> NumPy
tensor_data = torch.tensor([4.0, 5.0, 6.0])
np_from_tensor = tensor_data.numpy()  # 共享内存
print(f"\nTensor -> NumPy: {np_from_tensor}")

  2.1 张量与NumPy对比

2️⃣ Tensor与NumPy ndarray对比:

NumPy array:
[[1. 2. 3.]
 [4. 5. 6.]]

PyTorch tensor:
tensor([[1., 2., 3.],
        [4., 5., 6.]])

------------------------------------------------------------
常用操作对比:
------------------------------------------------------------
操作                   NumPy                     PyTorch                  
------------------------------------------------------------
创建零矩阵                np.zeros((2,3))           torch.zeros(2,3)         
创建随机矩阵               np.random.randn(2,3)      torch.randn(2,3)         
矩阵乘法                 A @ B 或 np.dot(A,B)       A @ B 或 torch.mm(A,B)    
逐元素乘法                A * B                     A * B                    
求和                   np.sum(A)                 torch.sum(A) 或 A.sum()   
形状                   A.shape                   A.shape 或 A.size()       
转置                   A.T                       A.T                      
reshape              A.reshape(3,2)            A.reshape(3,2) 或 view    
----

In [4]:
print("="*60)
print("  2.1 张量运算")
print("="*60)

# ============================================================
# 3. 张量运算：与NumPy几乎一致
# ============================================================
print("\n4️⃣ 张量运算:")

# 创建测试张量
a = torch.tensor([[1.0, 2.0], 
                  [3.0, 4.0]])
b = torch.tensor([[5.0, 6.0], 
                  [7.0, 8.0]])

print(f"a = \n{a}")
print(f"\nb = \n{b}")

# ----- 基本运算 -----
print("\n基本运算:")

# 加法
print(f"a + b = \n{a + b}")

# 逐元素乘法（Hadamard积）
print(f"\na * b (逐元素) = \n{a * b}")

# 矩阵乘法（最重要！神经网络的核心运算）
print(f"\na @ b (矩阵乘法) = \n{a @ b}")
# 等价于: torch.mm(a, b) 或 torch.matmul(a, b)

# ----- 统计运算 -----
print("\n统计运算:")
x = torch.tensor([[1.0, 2.0, 3.0],
                  [4.0, 5.0, 6.0]])
print(f"x = \n{x}")
print(f"\n总和: {x.sum()}")
print(f"均值: {x.mean()}")
print(f"最大值: {x.max()}")
print(f"按行求和 (dim=1): {x.sum(dim=1)}")
print(f"按列求和 (dim=0): {x.sum(dim=0)}")

# ----- 广播机制（与NumPy相同） -----
print("\n广播机制:")
matrix = torch.ones(2, 3)
vector = torch.tensor([1, 2, 3])
result = matrix + vector  # 自动广播
print(f"ones(2,3) + [1,2,3] = \n{result}")

  2.1 张量运算

4️⃣ 张量运算:
a = 
tensor([[1., 2.],
        [3., 4.]])

b = 
tensor([[5., 6.],
        [7., 8.]])

基本运算:
a + b = 
tensor([[ 6.,  8.],
        [10., 12.]])

a * b (逐元素) = 
tensor([[ 5., 12.],
        [21., 32.]])

a @ b (矩阵乘法) = 
tensor([[19., 22.],
        [43., 50.]])

统计运算:
x = 
tensor([[1., 2., 3.],
        [4., 5., 6.]])

总和: 21.0
均值: 3.5
最大值: 6.0
按行求和 (dim=1): tensor([ 6., 15.])
按列求和 (dim=0): tensor([5., 7., 9.])

广播机制:
ones(2,3) + [1,2,3] = 
tensor([[2., 3., 4.],
        [2., 3., 4.]])


In [5]:
print("="*60)
print("  2.1 张量形状操作")
print("="*60)

# ============================================================
# 4. 形状操作：reshape, view, transpose
# ============================================================
print("\n5️⃣ 形状操作:")

x = torch.arange(12)  # [0, 1, 2, ..., 11]
print(f"\n原始张量: {x}")
print(f"形状: {x.shape}")

# ----- reshape（改变形状） -----
x_3x4 = x.reshape(3, 4)
print(f"\nreshape(3,4):\n{x_3x4}")

# ----- view（类似reshape，但要求内存连续） -----
x_4x3 = x.view(4, 3)
print(f"\nview(4,3):\n{x_4x3}")

# ----- 使用-1自动推断维度 -----
x_auto = x.reshape(2, -1)  # 自动计算列数：12/2=6
print(f"\nreshape(2,-1) 自动推断:\n{x_auto}")

# ----- flatten（展平） -----
x_flat = x_3x4.flatten()
print(f"\nflatten(): {x_flat}")

# ----- transpose（转置） -----
print("\n转置操作:")
matrix = torch.tensor([[1, 2, 3],
                       [4, 5, 6]])
print(f"原矩阵 (2,3):\n{matrix}")
print(f"\n转置 .T (3,2):\n{matrix.T}")

# 高维张量的transpose
tensor_3d = torch.randn(2, 3, 4)
print(f"\n3D张量形状: {tensor_3d.shape}")
print(f"transpose(0,2)后: {tensor_3d.transpose(0, 2).shape}")
print(f"permute(2,0,1)后: {tensor_3d.permute(2, 0, 1).shape}")

  2.1 张量形状操作

5️⃣ 形状操作:

原始张量: tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])
形状: torch.Size([12])

reshape(3,4):
tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]])

view(4,3):
tensor([[ 0,  1,  2],
        [ 3,  4,  5],
        [ 6,  7,  8],
        [ 9, 10, 11]])

reshape(2,-1) 自动推断:
tensor([[ 0,  1,  2,  3,  4,  5],
        [ 6,  7,  8,  9, 10, 11]])

flatten(): tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

转置操作:
原矩阵 (2,3):
tensor([[1, 2, 3],
        [4, 5, 6]])

转置 .T (3,2):
tensor([[1, 4],
        [2, 5],
        [3, 6]])

3D张量形状: torch.Size([2, 3, 4])
transpose(0,2)后: torch.Size([4, 3, 2])
permute(2,0,1)后: torch.Size([4, 2, 3])


In [6]:
print("="*60)
print("  2.1 索引与切片 + GPU转移")
print("="*60)

# ============================================================
# 5. 索引和切片（与NumPy完全一致）
# ============================================================
print("\n6️⃣ 索引和切片:")

x = torch.tensor([[1, 2, 3],
                  [4, 5, 6],
                  [7, 8, 9]])
print(f"原始张量:\n{x}")

print(f"\nx[0, :] 第1行: {x[0, :]}")
print(f"x[:, 1] 第2列: {x[:, 1]}")
print(f"x[1:, 1:] 右下角2x2:\n{x[1:, 1:]}")
print(f"x[0, 2] 单个元素: {x[0, 2]}")

# 布尔索引
mask = x > 5
print(f"\n布尔掩码 x > 5:\n{mask}")
print(f"满足条件的元素: {x[mask]}")

# ============================================================
# 6. GPU/MPS设备转移
# ============================================================
print("\n" + "="*60)
print("7️⃣ 设备转移（GPU加速）:")

# 检测可用设备
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"使用设备: CUDA (NVIDIA GPU)")
elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    device = torch.device("mps")
    print(f"使用设备: MPS (Apple Silicon GPU)")
else:
    device = torch.device("cpu")
    print(f"使用设备: CPU")

# 创建CPU张量
x_cpu = torch.randn(3, 3)
print(f"\n原始张量设备: {x_cpu.device}")

# 转移到GPU/MPS
x_gpu = x_cpu.to(device)
print(f"转移后设备: {x_gpu.device}")

# 在GPU上运算
y_gpu = x_gpu @ x_gpu.T
print(f"GPU上计算结果设备: {y_gpu.device}")

# 转回CPU（例如用于绘图或NumPy操作）
y_cpu = y_gpu.cpu()
print(f"转回CPU: {y_cpu.device}")

# 转换为NumPy（必须先转到CPU！）
y_numpy = y_cpu.numpy()
print(f"\n⚠️ 注意: GPU张量必须先.cpu()才能.numpy()")

  2.1 索引与切片 + GPU转移

6️⃣ 索引和切片:
原始张量:
tensor([[1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]])

x[0, :] 第1行: tensor([1, 2, 3])
x[:, 1] 第2列: tensor([2, 5, 8])
x[1:, 1:] 右下角2x2:
tensor([[5, 6],
        [8, 9]])
x[0, 2] 单个元素: 3

布尔掩码 x > 5:
tensor([[False, False, False],
        [False, False,  True],
        [ True,  True,  True]])
满足条件的元素: tensor([6, 7, 8, 9])

7️⃣ 设备转移（GPU加速）:
使用设备: MPS (Apple Silicon GPU)

原始张量设备: cpu
转移后设备: mps:0
GPU上计算结果设备: mps:0
转回CPU: cpu

⚠️ 注意: GPU张量必须先.cpu()才能.numpy()


---

## 2.2 自动微分（Autograd）：告别手写反向传播！

### 🎯 这是PyTorch最核心的功能

在第4章，我们手写了痛苦的反向传播代码。现在，PyTorch可以**自动完成**！

### 工作原理

1. 设置 `requires_grad=True`：告诉PyTorch要跟踪这个张量的操作
2. 执行前向计算：PyTorch在后台构建**计算图**
3. 调用 `.backward()`：自动计算所有梯度
4. 读取 `.grad` 属性：获取计算好的梯度

### 对比

| 步骤 | NumPy手写 | PyTorch Autograd |
|------|-----------|------------------|
| 定义函数 | 写代码 | 写代码 |
| 推导梯度 | 手动用笔推导公式 | **不需要** |
| 实现梯度 | 写代码实现公式 | **调用.backward()** |
| 验证正确性 | 数值梯度检验 | **框架保证正确** |

In [7]:
print("="*60)
print("  2.2 自动微分基础")
print("="*60)

# ============================================================
# 示例1：简单函数的梯度计算
# ============================================================
print("\n示例1: 计算 f(x) = x² 在 x=3 处的梯度")
print("      数学推导: df/dx = 2x = 2×3 = 6")

# 步骤1: 创建张量，设置requires_grad=True
# 这告诉PyTorch：请跟踪对这个张量的所有操作
x = torch.tensor(3.0, requires_grad=True)
print(f"\n步骤1: x = {x}")
print(f"       requires_grad = {x.requires_grad}")

# 步骤2: 前向计算
# PyTorch在后台记录：y是x的平方
y = x ** 2
print(f"\n步骤2: y = x² = {y}")
print(f"       grad_fn = {y.grad_fn}")  # 记录了如何计算的

# 步骤3: 反向传播（计算梯度）
y.backward()
print(f"\n步骤3: 调用 y.backward()")

# 步骤4: 读取梯度
print(f"\n步骤4: x.grad = {x.grad}")
print(f"✅ 结果正确！dy/dx = 2×3 = 6")

  2.2 自动微分基础

示例1: 计算 f(x) = x² 在 x=3 处的梯度
      数学推导: df/dx = 2x = 2×3 = 6

步骤1: x = 3.0
       requires_grad = True

步骤2: y = x² = 9.0
       grad_fn = <PowBackward0 object at 0x103fbfee0>

步骤3: 调用 y.backward()

步骤4: x.grad = 6.0
✅ 结果正确！dy/dx = 2×3 = 6


In [8]:
print("="*60)
print("  2.2 多变量函数的梯度")
print("="*60)

# ============================================================
# 示例2：多变量函数
# ============================================================
print("\n示例2: 计算 f(x,y) = x² + 2xy + y² 的梯度")
print("      ∂f/∂x = 2x + 2y")
print("      ∂f/∂y = 2x + 2y")
print("      在(x=2, y=3)处: ∂f/∂x = ∂f/∂y = 2×2+2×3 = 10")

# 创建变量
x = torch.tensor(2.0, requires_grad=True)
y = torch.tensor(3.0, requires_grad=True)

# 前向计算
f = x**2 + 2*x*y + y**2
print(f"\nf(2,3) = {f.item()}")

# 反向传播
f.backward()

# 读取梯度
print(f"\n∂f/∂x = {x.grad.item()} (理论值: 10)")
print(f"∂f/∂y = {y.grad.item()} (理论值: 10)")
print("\n✅ 多变量函数的梯度也自动计算！")

  2.2 多变量函数的梯度

示例2: 计算 f(x,y) = x² + 2xy + y² 的梯度
      ∂f/∂x = 2x + 2y
      ∂f/∂y = 2x + 2y
      在(x=2, y=3)处: ∂f/∂x = ∂f/∂y = 2×2+2×3 = 10

f(2,3) = 25.0

∂f/∂x = 10.0 (理论值: 10)
∂f/∂y = 10.0 (理论值: 10)

✅ 多变量函数的梯度也自动计算！


In [9]:
print("="*60)
print("  2.2 对比：手写链式法则 vs Autograd")
print("="*60)

# ============================================================
# 示例3：复合函数（展示链式法则的自动应用）
# ============================================================
print("\n示例3: 复合函数 f(x) = sin(x²) 在 x=1 处的梯度")
print("")
print("手动推导链式法则:")
print("  设 u = x²")
print("  f = sin(u)")
print("  df/dx = df/du × du/dx")
print("        = cos(u) × 2x")
print("        = cos(x²) × 2x")
print("        = cos(1) × 2 ≈ 1.0806")

# ----- 手写计算 -----
import math
x_val = 1.0
manual_grad = math.cos(x_val**2) * 2 * x_val
print(f"\n手动计算: {manual_grad:.6f}")

# ----- PyTorch自动计算 -----
x = torch.tensor(1.0, requires_grad=True)
f = torch.sin(x**2)
f.backward()
print(f"PyTorch自动: {x.grad.item():.6f}")

print("\n✅ 完全一致！PyTorch自动应用了链式法则！")
print("\n" + "-"*60)
print("关键优势:")
print("  - 无需手动推导: cos(x²)×2x")
print("  - 无需担心出错")
print("  - 任意复杂的复合函数都能自动处理")

  2.2 对比：手写链式法则 vs Autograd

示例3: 复合函数 f(x) = sin(x²) 在 x=1 处的梯度

手动推导链式法则:
  设 u = x²
  f = sin(u)
  df/dx = df/du × du/dx
        = cos(u) × 2x
        = cos(x²) × 2x
        = cos(1) × 2 ≈ 1.0806

手动计算: 1.080605
PyTorch自动: 1.080605

✅ 完全一致！PyTorch自动应用了链式法则！

------------------------------------------------------------
关键优势:
  - 无需手动推导: cos(x²)×2x
  - 无需担心出错
  - 任意复杂的复合函数都能自动处理


In [10]:
print("="*60)
print("  2.2 模拟神经网络层的自动求导")
print("="*60)

# ============================================================
# 示例4：模拟神经网络的前向和反向传播
# ============================================================
print("\n示例4: 模拟线性层 y = Wx + b")
print("")
print("这正是神经网络的核心运算！")
print("PyTorch会自动计算 ∂L/∂W 和 ∂L/∂b")

# 输入数据（不需要梯度）
x = torch.tensor([1.0, 2.0, 3.0])
print(f"\n输入 x: {x}")

# 模型参数（需要梯度！）
W = torch.randn(2, 3, requires_grad=True)  # 权重矩阵
b = torch.randn(2, requires_grad=True)      # 偏置向量
print(f"权重 W 形状: {W.shape}")
print(f"偏置 b 形状: {b.shape}")

# 前向传播
y = W @ x + b  # 线性变换
print(f"\n输出 y = Wx + b: {y}")

# 假设损失函数是输出的和（简化示例）
loss = y.sum()
print(f"损失 L = sum(y): {loss.item():.4f}")

# 反向传播
loss.backward()

# 查看梯度
print(f"\n自动计算的梯度:")
print(f"  ∂L/∂W:\n{W.grad}")
print(f"\n  ∂L/∂b: {b.grad}")

print("\n" + "-"*60)
print("对比NumPy实现:")
print("  NumPy: 需要手动推导 dL/dW = x^T @ dL/dy")
print("  PyTorch: loss.backward() 一行搞定！")

  2.2 模拟神经网络层的自动求导

示例4: 模拟线性层 y = Wx + b

这正是神经网络的核心运算！
PyTorch会自动计算 ∂L/∂W 和 ∂L/∂b

输入 x: tensor([1., 2., 3.])
权重 W 形状: torch.Size([2, 3])
偏置 b 形状: torch.Size([2])

输出 y = Wx + b: tensor([-3.8251,  0.4256], grad_fn=<AddBackward0>)
损失 L = sum(y): -3.3995

自动计算的梯度:
  ∂L/∂W:
tensor([[1., 2., 3.],
        [1., 2., 3.]])

  ∂L/∂b: tensor([1., 1.])

------------------------------------------------------------
对比NumPy实现:
  NumPy: 需要手动推导 dL/dW = x^T @ dL/dy
  PyTorch: loss.backward() 一行搞定！


In [11]:
print("="*60)
print("  2.2 计算图可视化理解")
print("="*60)

# ============================================================
# 计算图的概念解释（文字可视化）
# ============================================================
print("""
计算图示例: y = (a + b) × c

前向传播（构建计算图）:
════════════════════════════════════════════════════════

    [a=2]  [b=3]                    [c=4]
       \    /                          |
        \  /                           |
       [Add]  ──────────────────>  [Mul]
         |                            |
      [t=5]                        [y=20]

    操作记录: t = a + b, y = t × c

════════════════════════════════════════════════════════

反向传播（自动微分）:
════════════════════════════════════════════════════════

    dy/dy = 1（从输出开始）
       ↓
    dy/dt = c = 4（乘法的梯度）
    dy/dc = t = 5
       ↓
    dy/da = dy/dt × dt/da = 4 × 1 = 4（链式法则）
    dy/db = dy/dt × dt/db = 4 × 1 = 4

════════════════════════════════════════════════════════
""")

# 用代码验证
print("代码验证:")
a = torch.tensor(2.0, requires_grad=True)
b = torch.tensor(3.0, requires_grad=True)
c = torch.tensor(4.0, requires_grad=True)

t = a + b
y = t * c

y.backward()

print(f"  y = (a+b)×c = (2+3)×4 = {y.item()}")
print(f"  dy/da = {a.grad.item()} (理论值: c = 4)")
print(f"  dy/db = {b.grad.item()} (理论值: c = 4)")
print(f"  dy/dc = {c.grad.item()} (理论值: a+b = 5)")
print("\n✅ PyTorch自动完成了链式法则的计算！")

  2.2 计算图可视化理解

计算图示例: y = (a + b) × c

前向传播（构建计算图）:
════════════════════════════════════════════════════════

    [a=2]  [b=3]                    [c=4]
       \    /                          |
        \  /                           |
       [Add]  ──────────────────>  [Mul]
         |                            |
      [t=5]                        [y=20]

    操作记录: t = a + b, y = t × c

════════════════════════════════════════════════════════

反向传播（自动微分）:
════════════════════════════════════════════════════════

    dy/dy = 1（从输出开始）
       ↓
    dy/dt = c = 4（乘法的梯度）
    dy/dc = t = 5
       ↓
    dy/da = dy/dt × dt/da = 4 × 1 = 4（链式法则）
    dy/db = dy/dt × dt/db = 4 × 1 = 4

════════════════════════════════════════════════════════

代码验证:
  y = (a+b)×c = (2+3)×4 = 20.0
  dy/da = 4.0 (理论值: c = 4)
  dy/db = 4.0 (理论值: c = 4)
  dy/dc = 5.0 (理论值: a+b = 5)

✅ PyTorch自动完成了链式法则的计算！


In [12]:
print("="*60)
print("  2.2 Autograd注意事项")
print("="*60)

# ============================================================
# 重要的使用注意事项
# ============================================================

print("\n⚠️ 注意事项1: 梯度会累积")
print("-"*40)

x = torch.tensor(2.0, requires_grad=True)

# 第一次计算
y1 = x ** 2
y1.backward()
print(f"第一次backward后: x.grad = {x.grad}")

# 第二次计算（不清零）
y2 = x ** 2
y2.backward()
print(f"第二次backward后: x.grad = {x.grad} (累积了！)")

# 正确做法：每次backward前清零
x.grad.zero_()  # 就地操作清零
y3 = x ** 2
y3.backward()
print(f"清零后再backward: x.grad = {x.grad} (正确！)")

print("\n⚠️ 注意事项2: 推理时禁用梯度（节省内存）")
print("-"*40)

x = torch.randn(1000, 1000, requires_grad=True)

# 训练时：需要梯度
y_train = x @ x.T
print(f"训练模式 - grad_fn存在: {y_train.grad_fn is not None}")

# 推理时：使用torch.no_grad()禁用梯度
with torch.no_grad():
    y_eval = x @ x.T
    print(f"推理模式 - grad_fn存在: {y_eval.grad_fn is not None}")

print("\n💡 关键点:")
print("  - 训练循环中: 保持requires_grad=True")
print("  - 推理/验证时: 使用with torch.no_grad():")
print("  - 每个batch: 用optimizer.zero_grad()清零梯度")

  2.2 Autograd注意事项

⚠️ 注意事项1: 梯度会累积
----------------------------------------
第一次backward后: x.grad = 4.0
第二次backward后: x.grad = 8.0 (累积了！)
清零后再backward: x.grad = 4.0 (正确！)

⚠️ 注意事项2: 推理时禁用梯度（节省内存）
----------------------------------------
训练模式 - grad_fn存在: True
推理模式 - grad_fn存在: False

💡 关键点:
  - 训练循环中: 保持requires_grad=True
  - 推理/验证时: 使用with torch.no_grad():
  - 每个batch: 用optimizer.zero_grad()清零梯度


---

## 2.3 nn.Module：定义神经网络模型

### 🏗️ 模型定义的核心类

`torch.nn.Module` 是所有神经网络的基类。

### 定义模型的两种方式

| 方式 | 适用场景 | 灵活性 |
|------|----------|--------|
| **继承nn.Module** | 任意复杂模型 | ⭐⭐⭐⭐⭐ |
| **nn.Sequential** | 简单顺序模型 | ⭐⭐⭐ |

### 核心步骤

1. 继承 `nn.Module`
2. 在 `__init__` 中定义层
3. 在 `forward` 中定义前向传播逻辑
4. **不需要定义backward！** (Autograd自动处理)

In [13]:
import torch.nn as nn
import torch.nn.functional as F

print("="*60)
print("  2.3 使用nn.Module定义MLP")
print("="*60)

# ============================================================
# 方式1：继承nn.Module（推荐，最灵活）
# ============================================================

class PyTorchMLP(nn.Module):
    """
    PyTorch实现的多层感知机
    
    结构: 输入层 -> 隐藏层(ReLU) -> 输出层
    
    对比NumPy实现的优势:
    1. 无需手动初始化权重（nn.Linear自动处理）
    2. 无需手动实现激活函数（F.relu直接调用）
    3. 无需实现backward（Autograd自动处理）
    """
    
    def __init__(self, input_size, hidden_size, output_size):
        """
        初始化网络层
        
        参数:
            input_size: 输入特征维度
            hidden_size: 隐藏层神经元数
            output_size: 输出类别数
        """
        # 必须调用父类初始化！
        super().__init__()
        
        # 定义网络层
        # nn.Linear: 全连接层，自动初始化权重和偏置
        # 等价于NumPy中的: W @ x + b
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)
        
        # 注意：不需要手动初始化权重！
        # nn.Linear默认使用Kaiming初始化
    
    def forward(self, x):
        """
        前向传播
        
        参数:
            x: 输入张量, shape (batch_size, input_size)
        
        返回:
            输出张量, shape (batch_size, output_size)
            
        注意：
        - 不需要保存中间变量！Autograd会自动记录
        - 不需要实现backward！Autograd会自动计算
        """
        # 第一层 + ReLU激活
        x = self.fc1(x)      # 线性变换
        x = F.relu(x)        # ReLU激活
        
        # 第二层（输出层不加激活，损失函数会处理）
        x = self.fc2(x)
        
        return x


# 实例化模型
model = PyTorchMLP(input_size=784, hidden_size=128, output_size=10)

print("\n模型结构:")
print(model)

# 查看参数
print("\n\n模型参数:")
for name, param in model.named_parameters():
    print(f"  {name}: 形状 {param.shape}")

# 计算总参数量
total_params = sum(p.numel() for p in model.parameters())
print(f"\n总参数数量: {total_params:,}")
print(f"  = 784×128 + 128 + 128×10 + 10")
print(f"  = {784*128} + {128} + {128*10} + {10}")
print(f"  = {784*128 + 128 + 128*10 + 10:,}")

  2.3 使用nn.Module定义MLP

模型结构:
PyTorchMLP(
  (fc1): Linear(in_features=784, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)


模型参数:
  fc1.weight: 形状 torch.Size([128, 784])
  fc1.bias: 形状 torch.Size([128])
  fc2.weight: 形状 torch.Size([10, 128])
  fc2.bias: 形状 torch.Size([10])

总参数数量: 101,770
  = 784×128 + 128 + 128×10 + 10
  = 100352 + 128 + 1280 + 10
  = 101,770


In [14]:
print("="*60)
print("  2.3 测试前向传播")
print("="*60)

# 创建假数据测试
batch_size = 16
fake_input = torch.randn(batch_size, 784)  # 模拟MNIST图像
print(f"\n输入形状: {fake_input.shape}")
print(f"  (batch_size=16, features=784)")

# 前向传播
output = model(fake_input)
print(f"\n输出形状: {output.shape}")
print(f"  (batch_size=16, num_classes=10)")

# 查看输出（未经过softmax，是logits）
print(f"\n第一个样本的输出（logits）:")
print(f"  {output[0].detach().numpy()}")

# 转换为概率
probs = F.softmax(output, dim=1)
print(f"\n第一个样本的概率:")
print(f"  {probs[0].detach().numpy()}")
print(f"  概率和: {probs[0].sum().item():.4f}")

print("\n✅ 前向传播成功！")

  2.3 测试前向传播

输入形状: torch.Size([16, 784])
  (batch_size=16, features=784)

输出形状: torch.Size([16, 10])
  (batch_size=16, num_classes=10)

第一个样本的输出（logits）:
  [-0.47005534 -0.53800935  0.28363952 -0.22265495 -0.09798245 -0.32043105
  0.00300348 -0.23516689  0.08927251 -0.15305798]

第一个样本的概率:
  [0.07171478 0.06700338 0.1523822  0.09184454 0.10403942 0.0832894
 0.11509476 0.09070254 0.12546475 0.09846433]
  概率和: 1.0000

✅ 前向传播成功！


In [15]:
print("="*60)
print("  2.3 nn.Sequential简化写法")
print("="*60)

# ============================================================
# 方式2：使用nn.Sequential（适合简单顺序模型）
# ============================================================

# 与上面的PyTorchMLP完全等价
model_sequential = nn.Sequential(
    nn.Linear(784, 128),   # 第一层
    nn.ReLU(),             # 激活函数
    nn.Linear(128, 10)     # 第二层
)

print("\nSequential模型结构:")
print(model_sequential)

# 测试前向传播
output_seq = model_sequential(fake_input)
print(f"\n输出形状: {output_seq.shape}")

print("\n" + "-"*60)
print("两种方式对比:")
print("-"*60)
print("""
nn.Module类（继承方式）:
  ✅ 最灵活，可以实现任意复杂的forward逻辑
  ✅ 可以有多个输入/输出
  ✅ 可以在forward中加入条件判断
  ⚠️ 代码稍多

nn.Sequential:
  ✅ 代码简洁
  ⚠️ 只能实现顺序堆叠的模型
  ⚠️ 不能有分支结构
  ⚠️ 不能有多个输入/输出
""")

  2.3 nn.Sequential简化写法

Sequential模型结构:
Sequential(
  (0): Linear(in_features=784, out_features=128, bias=True)
  (1): ReLU()
  (2): Linear(in_features=128, out_features=10, bias=True)
)

输出形状: torch.Size([16, 10])

------------------------------------------------------------
两种方式对比:
------------------------------------------------------------

nn.Module类（继承方式）:
  ✅ 最灵活，可以实现任意复杂的forward逻辑
  ✅ 可以有多个输入/输出
  ✅ 可以在forward中加入条件判断
  ⚠️ 代码稍多

nn.Sequential:
  ✅ 代码简洁
  ⚠️ 只能实现顺序堆叠的模型
  ⚠️ 不能有分支结构
  ⚠️ 不能有多个输入/输出



In [ ]:
print("="*60)
print("  2.3 核心对比：NumPy MLP vs PyTorch MLP")
print("="*60)

# ============================================================
# 并排对比：NumPy实现 vs PyTorch实现
# ============================================================

print("""
╔══════════════════════════════════════════════════════════════════╗
║                    NumPy实现 vs PyTorch实现                       ║
╠══════════════════════════════════════════════════════════════════╣
║                                                                   ║
║  【初始化权重】                                                    ║
║  ─────────────────────────────────────────────────────────────── ║
║  NumPy:                         │ PyTorch:                        ║
║  W1 = np.random.randn(784,128)  │ self.fc1 = nn.Linear(784, 128)  ║
║       * np.sqrt(2/784)          │ # 自动Xavier初始化！            ║
║  b1 = np.zeros(128)             │                                 ║
║  W2 = np.random.randn(128,10)   │ self.fc2 = nn.Linear(128, 10)   ║
║       * np.sqrt(2/128)          │                                 ║
║  b2 = np.zeros(10)              │                                 ║
║                                                                   ║
╠══════════════════════════════════════════════════════════════════╣
║                                                                   ║
║  【前向传播】                                                      ║
║  ─────────────────────────────────────────────────────────────── ║
║  NumPy:                         │ PyTorch:                        ║
║  self.z1 = X @ W1 + b1          │ x = self.fc1(x)                 ║
║  self.a1 = np.maximum(0, z1)    │ x = F.relu(x)                   ║
║  self.z2 = a1 @ W2 + b2         │ x = self.fc2(x)                 ║
║  # 必须保存中间变量！            │ # 不需要保存！                   ║
║                                                                   ║
╠══════════════════════════════════════════════════════════════════╣
║                                                                   ║
║  【反向传播】                                                      ║
║  ─────────────────────────────────────────────────────────────── ║
║  NumPy (30+行代码):             │ PyTorch:                        ║
║  dz2 = y_pred - y_true          │                                 ║
║  dW2 = a1.T @ dz2 / m           │ loss.backward()                 ║
║  db2 = np.sum(dz2, axis=0)/m    │ # 一行搞定！                     ║
║  da1 = dz2 @ W2.T               │                                 ║
║  dz1 = da1 * (z1 > 0)           │                                 ║
║  dW1 = X.T @ dz1 / m            │                                 ║
║  db1 = np.sum(dz1, axis=0)/m    │                                 ║
║                                                                   ║
╠══════════════════════════════════════════════════════════════════╣
║                                                                   ║
║  【参数更新】                                                      ║
║  ─────────────────────────────────────────────────────────────── ║
║  NumPy:                         │ PyTorch:                        ║
║  W1 -= lr * dW1                 │ optimizer.step()                ║
║  b1 -= lr * db1                 │ # 一行搞定所有参数！             ║
║  W2 -= lr * dW2                 │                                 ║
║  b2 -= lr * db2                 │                                 ║
║                                                                   ║
╚══════════════════════════════════════════════════════════════════╝
""")

print("代码行数对比:")
print("  NumPy MLP: ~80行（含backward、SGD、Adam等）")
print("  PyTorch MLP: ~15行")
print("\n开发效率: PyTorch提升约5倍！")

In [16]:
print("="*60)
print("  2.3 更复杂的模型示例")
print("="*60)

# ============================================================
# 示例：多隐藏层 + Dropout + BatchNorm
# ============================================================

class AdvancedMLP(nn.Module):
    """
    更复杂的MLP，包含:
    - 多个隐藏层
    - Batch Normalization（第7章学过）
    - Dropout正则化（第7章学过）
    
    结构: 784 -> 256 -> 128 -> 64 -> 10
    """
    
    def __init__(self, input_size=784, num_classes=10, dropout_rate=0.3):
        super().__init__()
        
        # 第一层: 784 -> 256
        self.fc1 = nn.Linear(input_size, 256)
        self.bn1 = nn.BatchNorm1d(256)  # Batch Normalization
        
        # 第二层: 256 -> 128
        self.fc2 = nn.Linear(256, 128)
        self.bn2 = nn.BatchNorm1d(128)
        
        # 第三层: 128 -> 64
        self.fc3 = nn.Linear(128, 64)
        self.bn3 = nn.BatchNorm1d(64)
        
        # 输出层: 64 -> 10
        self.fc4 = nn.Linear(64, num_classes)
        
        # Dropout层
        self.dropout = nn.Dropout(dropout_rate)
    
    def forward(self, x):
        # 第一层: Linear -> BN -> ReLU -> Dropout
        x = self.fc1(x)
        x = self.bn1(x)
        x = F.relu(x)
        x = self.dropout(x)
        
        # 第二层
        x = self.fc2(x)
        x = self.bn2(x)
        x = F.relu(x)
        x = self.dropout(x)
        
        # 第三层
        x = self.fc3(x)
        x = self.bn3(x)
        x = F.relu(x)
        x = self.dropout(x)
        
        # 输出层（无激活）
        x = self.fc4(x)
        
        return x


# 创建模型
advanced_model = AdvancedMLP()

print("\n高级MLP结构:")
print(advanced_model)

# 参数统计
total = sum(p.numel() for p in advanced_model.parameters())
trainable = sum(p.numel() for p in advanced_model.parameters() if p.requires_grad)
print(f"\n总参数: {total:,}")
print(f"可训练参数: {trainable:,}")

# 测试
advanced_model.eval()  # 评估模式（BN和Dropout行为不同）
test_input = torch.randn(8, 784)
test_output = advanced_model(test_input)
print(f"\n测试: 输入{test_input.shape} -> 输出{test_output.shape}")

  2.3 更复杂的模型示例

高级MLP结构:
AdvancedMLP(
  (fc1): Linear(in_features=784, out_features=256, bias=True)
  (bn1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=256, out_features=128, bias=True)
  (bn2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc3): Linear(in_features=128, out_features=64, bias=True)
  (bn3): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc4): Linear(in_features=64, out_features=10, bias=True)
  (dropout): Dropout(p=0.3, inplace=False)
)

总参数: 243,658
可训练参数: 243,658

测试: 输入torch.Size([8, 784]) -> 输出torch.Size([8, 10])


---

## 📝 本章小结

### 🎯 核心概念回顾

| 概念 | 说明 | 对比NumPy |
|------|------|----------|
| **Tensor** | 多维数组，支持GPU | 类似ndarray |
| **requires_grad** | 启用梯度跟踪 | 无此功能 |
| **backward()** | 自动计算梯度 | 需手写 |
| **nn.Module** | 模型基类 | 无此概念 |
| **nn.Linear** | 全连接层 | 手写W@x+b |

### ✅ 学习检查点

完成本章后，你应该能够：

- [ ] 理解为什么需要深度学习框架
- [ ] 创建和操作PyTorch Tensor
- [ ] 理解Autograd自动微分的工作原理
- [ ] 使用nn.Module定义神经网络模型
- [ ] 对比NumPy实现和PyTorch实现的区别

---

### 🚀 下一章预告

**10b: PyTorch完整训练**

- 损失函数与优化器（对比手写SGD/Adam）
- DataLoader数据加载
- 完整训练循环
- 模型保存与加载
- 实战：鸢尾花分类、手写数字识别